<a href="https://colab.research.google.com/github/Kalit31/IR-Assignment/blob/main/test_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>